<a href="https://colab.research.google.com/github/bas10agu/CSOPESY_GROUP2/blob/main/DeepDive_CUDA_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Group 2 - DeepDive CUDA - Histogram counting (Atomic operations)**
## **GROUP 2 - S11**

**MEMBERS:**

- Alfred Bastin S. Agustines
- Allan David C. De Leon
- Michael Angelo Depasucat
- Kai Hiori J. Padilla


# (1) C Histogram program

In [ ]:
%%writefile C_histo.c
#include <stdio.h>
#include <stdlib.h>
#include <stdatomic.h>
#include <time.h>
#include <stdint.h>

#define NUM_BINS 10

void getHistogram(int* vector, int vectorSize, atomic_int* histogram) {
    for (int i = 0; i < vectorSize; i++) {
        int bIndex = vector[i] % NUM_BINS;
        atomic_fetch_add(&histogram[bIndex], 1);
    }
}

int main() {
    const size_t ARRAY_SIZE = 1<<24;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(double);

    int32_t *vector;
    vector = (int32_t*)malloc(ARRAY_BYTES);
    int histogram[NUM_BINS] = { 0 };
    const size_t loope = 100;

    for (int i = 0; i < ARRAY_SIZE; i++) {
        vector[i] = i;
    }

    clock_t start, end;
    getHistogram(vector, ARRAY_SIZE, histogram);


    for (int i = 0; i < 10; i++) {
        histogram[i] = 0;
    }

    double elapse, time_taken;
    elapse = 0.0f;
    for (int i = 0; i < loope; i++) {
        for (int i = 0; i < 10; i++) {
            histogram[i] = 0;
        }
        start = clock();
        getHistogram(vector, ARRAY_SIZE, histogram);
        end = clock();
        time_taken = ((double)(end - start)) * 1E3 / CLOCKS_PER_SEC;
        elapse = elapse + time_taken;
    }
    printf("Function (in C) average time for %lu loops is %f milliseconds to execute an array size %lu \n", loope, elapse / loope, ARRAY_SIZE);

    size_t err_count = 0;
    int32_t  test[NUM_BINS] = { 0 };

    for (int i = 0; i < ARRAY_SIZE; i++) {
        int bIndex = vector[i] % 10;
        test[bIndex]++;
    }
    for (int i = 0; i < 10; i++) {

        if (test[i] != histogram[i])
            err_count++;
    }
    printf("Error count (C program): %lu\n", err_count);

    for (int i = 0; i < NUM_BINS; i++) {
        printf("Bin %d: %d\n", i, histogram[i]);
    }

    free(vector);
	return 0;
}

Overwriting C_histo.c


In [ ]:
%%shell
gcc C_histo.c -o C_histo

In [ ]:
%%shell
./C_histo

Function (in C) average time for 100 loops is 77.835400 milliseconds to execute an array size 16777216 
Error count (C program): 0
Bin 0: 1677722
Bin 1: 1677722
Bin 2: 1677722
Bin 3: 1677722
Bin 4: 1677722
Bin 5: 1677722
Bin 6: 1677721
Bin 7: 1677721
Bin 8: 1677721
Bin 9: 1677721


# (2) CUDA program w/ Unified memory, pre-fetching and memadvise

In [ ]:
%%writefile cuda.cu
#include <stdio.h>
#include <stdlib.h>
#include <cuda_runtime.h>
#include <chrono>

#define VECTOR_SIZE (1 << 24)
#define NUM_BINS 10

// CUDA kernel using atomic operations
__global__ void histogramKernel(int *d_data, int *d_histogram, int size) {
    int index = blockIdx.x * blockDim.x + threadIdx.x;
    int stride = blockDim.x * gridDim.x;
    for (int i = index; i < size; i += stride) {
        int bin = d_data[i] % NUM_BINS;
        atomicAdd(&d_histogram[bin], 1);
    }
}

int main() {
    const size_t ARRAY_SIZE = VECTOR_SIZE;
    const size_t ARRAY_BYTES = ARRAY_SIZE * sizeof(int);
    const size_t loope = 100;

    // Declare array
    int *data, *histogram;
    cudaMallocManaged(&data, ARRAY_BYTES);
    cudaMallocManaged(&histogram, NUM_BINS * sizeof(int));

    // Get GPU device
    int device = -1;
    cudaGetDevice(&device);

    // Memory advise
    cudaMemAdvise(data, ARRAY_BYTES, cudaMemAdviseSetPreferredLocation, cudaCpuDeviceId);
    cudaMemAdvise(data, ARRAY_BYTES, cudaMemAdviseSetReadMostly, cudaCpuDeviceId);

    // Prefetch to create CPU page memory
    cudaMemPrefetchAsync(data, ARRAY_BYTES, cudaCpuDeviceId, NULL);
    cudaMemPrefetchAsync(histogram, NUM_BINS * sizeof(int), device, NULL);

    // Initialize array
    for (size_t i = 0; i < VECTOR_SIZE; i++) {
        data[i] = i;
    }

    // Prefetch data from CPU to GPU
    cudaMemPrefetchAsync(data, ARRAY_BYTES, device, NULL);

    // Setup CUDA kernel
    size_t numThreads = 1024;
    size_t numBlocks = (VECTOR_SIZE + numThreads - 1) / numThreads;

    printf("*** function = Histogram Counting\n");
    printf("numElements = %lu\n", ARRAY_SIZE);
    printf("numBlocks = %lu, numThreads = %lu \n", numBlocks, numThreads);

    for (size_t i = 0; i < loope; i++) {
        cudaMemset(histogram, 0, NUM_BINS * sizeof(int));
        histogramKernel<<<numBlocks, numThreads>>>(data, histogram, VECTOR_SIZE);
    }

    // Synchronize device
    cudaDeviceSynchronize();

    // Prefetch result back to CPU
    cudaMemPrefetchAsync(histogram, NUM_BINS * sizeof(int), cudaCpuDeviceId, NULL);

    // Error checking
    size_t err_count = 0;
    int referenceHistogram[NUM_BINS] = {0};
    for (size_t i = 0; i < VECTOR_SIZE; i++) {
        int bin = data[i] % NUM_BINS;
        referenceHistogram[bin]++;
    }
    for (int i = 0; i < NUM_BINS; i++) {
        if (histogram[i] != referenceHistogram[i]) {
            printf("Mismatch at bin %d: CUDA=%d, CPU=%d\n", i, histogram[i], referenceHistogram[i]);
            err_count++;
        }
    }
    printf("Error count (CUDA program): %zu\n", err_count);

    // Print results
    for (int i = 0; i < NUM_BINS; i++) {
        printf("Bin %d: %d\n", i, histogram[i]);
    }

    // Free memory
    cudaFree(data);
    cudaFree(histogram);

    return 0;
}

Writing cuda.cu


In [ ]:
%%shell
nvcc cuda.cu -o cuda

In [ ]:
%%shell
./cuda

*** function = Histogram Counting
numElements = 16777216
numBlocks = 16384, numThreads = 1024 
Error count (CUDA program): 0
Bin 0: 1677722
Bin 1: 1677722
Bin 2: 1677722
Bin 3: 1677722
Bin 4: 1677722
Bin 5: 1677722
Bin 6: 1677721
Bin 7: 1677721
Bin 8: 1677721
Bin 9: 1677721
